 # MVPA Full 9 emotions


In [1]:
import numpy as np
import nibabel as nb
import os
from nilearn.input_data import NiftiMasker

In [2]:
# define paths
root_dir = '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2'
fmriprep_dir = os.path.join(root_dir, 'derivatives', 'fmriprep23')
dataset_dir  = os.path.join(root_dir, 'derivatives', 'mvpa_03_full')

# brain masks
#mask_brain_file = os.path.join(root_dir, 'derivatives', 'mni_icbm152_t1_tal_nlin_asym_09c.nii')
mask_gm_file    = os.path.join(root_dir, 'derivatives', 'mni_icbm152_gm_tal_nlin_asym_09c_res-2_dilated.nii')
mask_stab_file  = os.path.join(root_dir, 'derivatives', 'mvpa_03_full_temp', 'STAB_mask.nii.gz')

In [3]:
# list datasets and concatenate

# find all *_dataset.nii.gz files in dataset_dir
dataset_files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('_full_confounds_dataset.nii.gz') & f.startswith('sub-')]
dataset_files.sort()

# find all *_trial_types.txt files in dataset_dir
label_files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('_full_confounds_trial_types.txt') & f.startswith('sub-')]
label_files.sort()

In [4]:
# concatenate all datasets
D = nb.concat_images(dataset_files, axis=3)

In [5]:
# concatenate all labels into a single string array
labels = np.concatenate([np.loadtxt(l, dtype=str) for l in label_files])

# trim each label to remove the 2 digit number in the end
labels = np.array([l[:-2] for l in labels])

In [7]:
# Load mask
#masker = NiftiMasker(mask_img=mask_gm_file, standardize=False, detrend=False)
masker = NiftiMasker(mask_img=mask_stab_file, standardize=False, detrend=False)
samples = masker.fit_transform(D)

In [12]:
# generate an array of chunk labels
# 13 subjects, 9 x 2 music for each of the 4 runs
chunks = np.repeat(np.arange(1,14), 9*2*4)

In [13]:
# Let's specify the classifier
from sklearn.svm import LinearSVC
clf = LinearSVC(multi_class="ovr", max_iter=1000, class_weight='balanced')


In [14]:
# Perform the cross validation (takes time to compute)
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
cv_scores = cross_val_score(estimator=clf,
                            X=samples,
                            y=labels,
                            groups=chunks,
                            cv=LeaveOneGroupOut(),
                            n_jobs=6,
                            verbose=1)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/envs/brainplayback_task02/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual

In [15]:
print('Average accuracy = %.02f percent\n' % (cv_scores.mean() * 100))
print('Accuracy per fold:', cv_scores, sep='\n')


Average accuracy = 13.25 percent

Accuracy per fold:
[0.18055556 0.125      0.13888889 0.08333333 0.125      0.13888889
 0.13888889 0.11111111 0.11111111 0.18055556 0.13888889 0.11111111
 0.13888889]
